In [8]:
# imports
import numpy as np
import krakenex as k
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
import datetime
from pathlib import Path
import sys
from datetime import datetime, timedelta
from matplotlib import dates 


#set SQL BBDD
from sqlalchemy import create_engine, MetaData, Table, Column, DateTime, Float, String,Integer

dbInstance1 = 'sqlite:///krakenTeader1.db'
dbInstance2 = 'sqlite:///krakenTeader2.db'

dbBalanceHistoryTable = 'BalanceHistory'
dbTradesHistoryTable = 'TradesHistory'
dbMyHistoryTable = 'MyTrades'
engine1 = create_engine(dbInstance1)
engine2 = create_engine(dbInstance2)

BalanceHistory = pd.read_sql(dbBalanceHistoryTable, con=engine1)
BalanceHistory = BalanceHistory.set_index(pd.DatetimeIndex(BalanceHistory['Time']))
BalanceHistory.drop('Time', axis=1,inplace=True)

tradesHistory = pd.read_sql(dbTradesHistoryTable, con=engine1)
tradesHistory.set_index(pd.DatetimeIndex(tradesHistory['time']),inplace=True)


BalanceHistory2 = pd.read_sql(dbBalanceHistoryTable, con=engine2)
BalanceHistory2 = BalanceHistory2.set_index(pd.DatetimeIndex(BalanceHistory2['Time']))
BalanceHistory2.drop('Time', axis=1,inplace=True)

BalanceHistory2 = BalanceHistory2[BalanceHistory2.index > BalanceHistory.last_valid_index()]

tradesHistory2 = pd.read_sql(dbTradesHistoryTable, con=engine2)
tradesHistory2.set_index(pd.DatetimeIndex(tradesHistory2['time']),inplace=True)

tradesHistory2 = tradesHistory2[tradesHistory2.index > tradesHistory.last_valid_index()]


bh =pd.concat([BalanceHistory,BalanceHistory2],axis=0)
td =pd.concat([tradesHistory,tradesHistory2],axis=0)


dbInstance = 'sqlite:///krakenTeader.db'
engine = create_engine(dbInstance)

if not engine.dialect.has_table(engine, dbBalanceHistoryTable):  # If table don't exist, Create.
    metadata = MetaData(engine)
    
    # Create a table with the appropriate Columns
    Table(dbBalanceHistoryTable, metadata,
          Column('Time', DateTime, primary_key=True, nullable=False), 
          Column('close', Float), 
          Column('ask', Float), 
          Column('bid', Float),
          Column('balanceRatio', Float),
          Column('volbuy', Float), 
          Column('volsell', Float),
          Column('unbalance', Float))
    # Implement the creation
    metadata.create_all() 
    
if not engine.dialect.has_table(engine, dbTradesHistoryTable):  # If table don't exist, Create.
    metadata = MetaData(engine)
    
    # Create a table with the appropriate Columns
    Table(dbTradesHistoryTable, metadata,
          Column('price', Float, nullable=False), 
          Column('buy_sell', String,  nullable=False),  
          Column('market_limit', String),
          Column('miscellaneous', String),
          Column('time', DateTime,  nullable=False),
          Column('volume', Float, nullable=False))
    # Implement the creation
    metadata.create_all()
    
bh.to_sql(dbBalanceHistoryTable,engine, if_exists='append')
td.drop('time', axis=1,inplace=True)
td.to_sql(dbTradesHistoryTable,engine, if_exists='append')

In [2]:
#set SQL BBDD
import numpy as np
import krakenex as k
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, DateTime, Float, String,Integer
from datetime import datetime, timedelta
# Convert a unix time u to a datetime object d
def unixtoDate(u): 
    return u  - timedelta(hours=1)

dbInstance1 = 'sqlite:///krakenTeader.db'
dbTradesHistoryTable = 'TradesHistory'
engine1 = create_engine(dbInstance1)

tradesHistory = pd.read_sql(dbTradesHistoryTable, con=engine1)
tradesHistory.set_index(pd.DatetimeIndex(tradesHistory['time']),inplace=True)

tradesHistory.tail()

,price,buy_sell,market_limit,miscellaneous,time,volume
time,,,,,,
2017-10-11 22:42:36,4850.0,b,m,,2017-10-11 22:42:36,0.007589
2017-10-11 22:42:40,4849.9,s,m,,2017-10-11 22:42:40,0.025000
2017-10-11 22:42:45,4850.0,b,l,,2017-10-11 22:42:45,0.020337
2017-10-11 22:43:51,4849.1,s,m,,2017-10-11 22:43:51,0.002100
2017-10-11 22:43:51,4849.0,s,m,,2017-10-11 22:43:51,0.049585


In [3]:
tradesHistory.tail()

,price,buy_sell,market_limit,miscellaneous,time,volume
time,,,,,,
2017-11-07 00:17:47.477100,7154.0,s,m,,2017-11-07 00:17:47.477100,0.045000
2017-11-07 00:18:04.431400,7161.4,s,l,,2017-11-07 00:18:04.431400,0.120000
2017-11-07 00:18:04.936700,7159.6,s,l,,2017-11-07 00:18:04.936700,0.005257
2017-11-07 00:18:10.322500,7169.9,b,m,,2017-11-07 00:18:10.322500,0.005000
2017-11-07 00:18:32.409800,7159.9,b,l,,2017-11-07 00:18:32.409800,0.002094


In [9]:
#corregir desface 1 hora
"""
import sqlite3
conn = sqlite3.connect('krakenTeader.db')
c = conn.cursor()
mydata = c.execute("UPDATE TradesHistory SET time=DATETIME(time, '-1 hours');")
conn.commit()
c.close
"""

<function Cursor.close>